In [108]:
cd /content/drive/MyDrive/Git/exercise/Code

/content/drive/MyDrive/Git/exercise/Code


# Module import 

In [109]:
import pandas as pd
import numpy as np

# Define function

In [110]:
def bmi_calc(x):
    return np.round(x['ITEM_F002'] / ((x['ITEM_F001']/100)**2),1)

In [129]:
def Grouping(df, group):
    df = df.copy()
    age_bins = [0,10,20,30,40,50,60,70,df.TEST_AGE.max()]
    age_labels = ['10세 미만','10대','20대','30대','40대','50대','60대','70대 이상']
    bmi_bins = [0,18.5,23,25,30,35,df.ITEM_F018.max()]
    bmi_labels = ['저체중','정상체중','비만전단계','1단계비만','2단계비만','3단계비만']

    df['AGE_GROUP'] = pd.cut(df['TEST_AGE'], bins=age_bins, labels=age_labels)
    df['BMI_GROUP'] = pd.cut(df['ITEM_F018'], bins=bmi_bins, labels=bmi_labels)
    df = df.rename(columns={'CERT_GBN':'CERT_GROUP'})
    ret_ = pd.merge(df,group,how='left').reset_index().rename(columns={'index':'USER_ID'})
    return ret_

# Prepare Data

## Load data

In [146]:
data = pd.read_csv('../Data/processed_data.csv')
user_group = pd.read_csv('../Data/user_group.csv')
sample_data = pd.read_csv('../Data/Sample_data.csv')

## Prepare sample user data

In [147]:
sample_data = sample_data.iloc[:300]
sample_data['ITEM_F018'] = sample_data.apply(bmi_calc,axis=1)
sample_data.head(5)

,TEST_AGE,CERT_GBN,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018
0,25,하,M,167.3,46.98,16.8
1,19,하,M,179.0,84.00,26.2
2,19,하,M,171.5,95.60,32.5
3,20,하,F,158.4,43.84,17.5
4,53,하,M,173.4,73.40,24.4


In [148]:
group_sample_data = Grouping(sample_data, user_group)
group_sample_data.head(5)

,USER_ID,TEST_AGE,CERT_GROUP,TEST_SEX,ITEM_F001,ITEM_F002,ITEM_F018,AGE_GROUP,BMI_GROUP,G_ID
0,0,25,하,M,167.3,46.98,16.8,20대,저체중,74
1,1,19,하,M,179.0,84.00,26.2,10대,1단계비만,56
2,2,19,하,M,171.5,95.60,32.5,10대,2단계비만,62
3,3,20,하,F,158.4,43.84,17.5,10대,저체중,41
4,4,53,하,M,173.4,73.40,24.4,50대,비만전단계,194


# Model